# PL

"""
Necessary folders and files
PRC_inputs:
    PL_inputs/plantsconfig1 and PL exe and SPORES exe
    target_DB_docking.sdf con columna IX y pasada por OBABEL.


PLANTS exclusively uses the MOL2 file format, thus MOL2-files (including bond connectivity)
must be provided for all input files. PLANTS expects correct MOL2-atom- and
bond-types. This is needed for the correct identification of rotatable bonds and charged
functional groups and may influence docking and virtual screening performance. Please
consider SPORES for this step.
"""

In [2]:
import numpy as np
import subprocess
from subprocess import Popen
from time import sleep
import os
import pandas as pd

In [1]:
#INPUTS
target='lck'
icm_command='/media/emilia/HDD/icm-browser-pro-3.9-4/icmbrowserpro64'
N_cores=7
cwd='/media/emilia/HDD'
PL_dir= cwd + '/Documents/PL/'
target_dir = PL_dir + target + '/'
rec_dir = target_dir + target + '_rec'

In [6]:
#ARCHIVOS
rec_mol2 = rec_dir + '.mol2' # generated with auto AD
mols_filename=target_dir+target+'_DB_docking.mol2' # generated with auto AD
#mols_filename=target_dir+target+'_new_p11_analogs.sdf' # no lo tengo por eso cambio el codigo de leer el archivo

In [7]:
"""
Lee el el archivo mols_ filename y averigua Nmol
el archivo mol2 no tiene la linea $$$$
"""
f=open(mols_filename,'r') # lo abre en modo lectura (r)
lines=f.readlines() # cada elemento en la lista corresponde a una linea en el archivo, lee todas la lineas
ix=[i for i in range(len(lines)) if lines[i]=='@<TRIPOS>MOLECULE\n'] #Esta línea crea una lista ix con las posiciones de las líneas que tiene @<TRIPOS>MOLECULE
ix.insert(0,-1) #agrega en el primer lugar un menos uno pq en esa linea se "encuntra" el primer $$$$
Nmols=len(ix)-1 # pq se agrego uno en el primer lugar algo
print('Number of molecules is ',Nmols)
f.close()


Number of molecules is  58


In [9]:
def edit_config(config_filename, N_cores, target, target_dir, rec_dir):
    fpl = open(config_filename, 'r')
    lines_fpl = fpl.readlines()
    fpl.close()

    rec_gpf = open(rec_dir + '.gpf', 'r')
    lines_rec_gpf = rec_gpf.readlines()
    rec_gpf.close()

# lines_rec_gpf[6] = gridcenter 6.27 0.998 -5.218 (cuando dice split ` ` divide por espacios)
    x_cent = (lines_rec_gpf[6].split(' ')[1])
    y_cent = (lines_rec_gpf[6].split(' ')[2])
    z_cent = (lines_rec_gpf[6].split(' ')[3])
    bs_rec = int(max(np.array(lines_rec_gpf[0].split(' ')[1:4]).astype(int)) / 2) # calcula la mitad del valor máximo entre los tres números encontrados en la primera línea

# preguntar a vale pq en esas lineas?
# si lo hiciera yo no deberia cerar un archivo config y escribir ahi? -->
    lines_fpl[6] = 'protein_file ' + target + '_rec.mol2 \n'
    lines_fpl[18] = 'bindingsite_center ' + x_cent + ' ' + y_cent + ' ' + z_cent + ' \n'
    lines_fpl[19] = 'bindingsite_radius ' + str(bs_rec) + ' \n'

    num = np.arange(N_cores + 1)[1::] #lo ultimo es un slicing, del uno en adelante
    # cambio y pongo el slicing a partir del 2 pq le pase el config 1 de vale y no tengo un config "normal"
    for i in num:
        lines_fpl[7] = 'ligand_file ' + target + '_' + str(i) + '.mol2 \n'
        lines_fpl[10] = 'output_dir results' + str(i) + ' \n'

        fpl = open(target_dir + '/plantsconfig' + str(i), 'w')
        for line in lines_fpl:
            fpl.write(line)
        fpl.close()

    return

In [12]:
edit_config(target_dir + 'plantsconfig1', N_cores, target, target_dir, rec_dir)
#chekear que se crearon y no estan vacios

FileNotFoundError: [Errno 2] No such file or directory: '/media/emilia/HDD/Documents/PL/p85/p85_rec.gpf'

In [14]:
"""
Generate N_cores files with mols and configs: 
"""
r=np.linspace(0,Nmols,N_cores+1,dtype='int')
# np.linspace divide Nmols en N_cores + 1 en partes iguales y devuelve un arreglo con esos valores
alphabet=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

for i in range(len(r) - 1):
    j0 = r[i] + 1 # la primer linea es cero o del anterior
    jf = r[i + 1]
    
#aca con openbabel por las dudas se pasa el file para que sea todo compatible con PLANTS
    proc = Popen(['/usr/bin/obabel', '-imol2', mols_filename, '-omol2', '-O', os.path.join(target_dir, target + '_' + alphabet[i] + '.mol2'), '-f', str(j0), '-l', str(jf)])
#-f se utiliza para especificar el número de inicio, -l el del final
# divide el archivo en BD_docking en N_cores en archivos target_alphabet.mol2

sleep(12) 

8 molecules converted
8 molecules converted
8 molecules converted
8 molecules converted
8 molecules converted
9 molecules converted
8 molecules converted


In [16]:
# corre SPORES y convierte file de alphabet a numeros
for i in range(len(r)-1):
    proc= Popen(['/media/emilia/HDD/Documents/PL/./SPORES_64bit','--mode','readbonds',target+'_'+alphabet[i]+'.mol2',target+'_'+str(i+1)+'.mol2'], cwd=target_dir)
sleep(5)




                                SPORES                                        SPORES                                        SPORES        







             Structure PrOtonation and REcognition System             Structure PrOtonation and REcognition System             Structure PrOtonation and REcognition System




                             version 1.28
                             version 1.28
                                SPORES                                     version 1.28

                                SPORES        






author: T. ten Brink
             Structure PrOtonation and REcognition System


author: T. ten Brink
             Structure PrOtonation and REcognition System
author: T. ten Brink


scientific contributors: P. Duchstein, J. Gimmler, O. Korb, M. Zerara,

                             version 1.28


scientific contributors: P. Duchstein, J. Gimmler, O. Korb, M. Zerara,
                             version 1.28                         T.E. Exnersci

In [19]:
num=np.arange(N_cores+1)[1::] #crea un arreglon con n enteros del 1 al Ncores, sin inculir el 0
for i in num:
    subprocess.Popen(['/media/emilia/HDD/Documents/PL/PLANTS1.2_64bit','--mode','screen',target_dir + str(i)+'/plantsconfig'+str(i),'>',target_dir+target+'_'+str(i)+'.log','&'])



                  PLANTS
        Protein-Ligand ANT System
               version 1.2


author: Oliver Korb

scientific contributors: T.E. Exner, T. Stuetzle

contact: Oliver.Korb@uni-konstanz.de


run PLANTS: PLANTS --mode screen yourconfigfile



                  PLANTS
        Protein-Ligand ANT System
               version 1.2


author: Oliver Korb

scientific contributors: T.E. Exner, T. Stuetzle

contact: Oliver.Korb@uni-konstanz.de


run PLANTS: PLANTS --mode screen yourconfigfile



                  PLANTS
        Protein-Ligand ANT System
               version 1.2


author: Oliver Korb

scientific contributors: T.E. Exner, T. Stuetzle

contact: Oliver.Korb@uni-konstanz.de


run PLANTS: PLANTS --mode screen yourconfigfile



                  PLANTS
        Protein-Ligand ANT System
               version 1.2


author: Oliver Korb

scientific contributors: T.E. Exner, T. Stuetzle

contact: Oliver.Korb@uni-konstanz.de


run PLANTS: PLANTS --mode screen yourconfigfile



  

PLANTS error: Cannot create output directory results1/

PLANTS error: Cannot create output directory results1/

PLANTS error: Cannot create output directory results1/

PLANTS error: Cannot create output directory results1/

PLANTS error: Cannot create output directory results1/

PLANTS error: Cannot create output directory results1/

PLANTS error: Cannot create output directory results1/



                  PLANTS
        Protein-Ligand ANT System
               version 1.2


author: Oliver Korb

scientific contributors: T.E. Exner, T. Stuetzle

contact: Oliver.Korb@uni-konstanz.de


run PLANTS: PLANTS --mode screen yourconfigfile



In [21]:
for i in num:
    subprocess.Popen(['/media/emilia/HDD/Documents/PL/PLANTS1.2_64bit','--mode','screen','/media/emilia/HDD/Documents/PL/p85/'+str(i)+'/plantsconfig'+str(i)])



                  PLANTS
        Protein-Ligand ANT System
               version 1.2


author: Oliver Korb

scientific contributors: T.E. Exner, T. Stuetzle

contact: Oliver.Korb@uni-konstanz.de


run PLANTS: PLANTS --mode screen yourconfigfile



                  PLANTS
        Protein-Ligand ANT System
               version 1.2


author: Oliver Korb

scientific contributors: T.E. Exner, T. Stuetzle

contact: Oliver.Korb@uni-konstanz.de


run PLANTS: PLANTS --mode screen yourconfigfile



                  PLANTS
        Protein-Ligand ANT System
               version 1.2


author: Oliver Korb

scientific contributors: T.E. Exner, T. Stuetzle

contact: Oliver.Korb@uni-konstanz.de


run PLANTS: PLANTS --mode screen yourconfigfile



                  PLANTS
        Protein-Ligand ANT System
               version 1.2


author: Oliver Korb

scientific contributors: T.E. Exner, T. Stuetzle

contact: Oliver.Korb@uni-konstanz.de


run PLANTS: PLANTS --mode screen yourconfigfile



  

PLANTS error: Cannot create output directory results1/

PLANTS error: Cannot create output directory results1/

PLANTS error: Cannot create output directory results1/

PLANTS error: Cannot create output directory results1/

PLANTS error: Cannot create output directory results1/

PLANTS error: Cannot create output directory results1/

PLANTS error: Cannot create output directory results1/





                  PLANTS
        Protein-Ligand ANT System
               version 1.2


author: Oliver Korb

scientific contributors: T.E. Exner, T. Stuetzle

contact: Oliver.Korb@uni-konstanz.de


run PLANTS: PLANTS --mode screen yourconfigfile



# # Process results
Run only when PL finished

In [3]:
import csv
def concatenate():
    # Crear el archivo 'bestranked_ligands' si no existe y añadir el encabezado una vez
    with open(target_dir + f'bestranked_ligands_{target}.csv', 'w', newline='') as dest_csv:
        writer = csv.writer(dest_csv)
        header_written = False

        for i in range(1, N_cores + 1):
            source_file = target_dir  + str(i) + "/results1/ranking/bestranking_ordered.csv"
            with open(source_file, 'r') as source_csv:
                source_data = list(csv.reader(source_csv))

                # Escribir el encabezado solo una vez
                if not header_written:
                    writer.writerow(source_data[0])  # Escribir encabezado
                    header_written = True
                writer.writerows(source_data[1:])  # Escribir datos sin encabezado

    print(os.path.exists(f'{target_dir}/bestranked_ligands_{target}.csv'))

    # Crear el archivo 'out_plants' si no existe y añadir el encabezado una vez
    with open(target_dir + f'/out_plants_{target}.mol2', 'w', newline='') as dest_csv:
        writer = csv.writer(dest_csv)
        header_written = False

        for i in range(1, N_cores + 1):
            source_file = target_dir  + str(i) + "/results1/ranking/ranked_ligands.mol2"
            with open(source_file, 'r') as source_csv:
                source_data = list(csv.reader(source_csv))

                # Escribir el encabezado solo una vez
                if not header_written:
                    writer.writerow(source_data[0])  # Escribir encabezado
                    header_written = True
                writer.writerows(source_data[1:])  # Escribir datos sin encabezado

concatenate()
sleep(3)


True


In [51]:
def concatenate_w_water():
    
    destination_file = target_dir + "bestranked_ligands_"+target+".csv"
    open(destination_file, 'w').close()

    # Run the command N_cores times
    for i in range(1, N_cores + 1):
        source_file = target_dir +  str (i) +  "/results1" + "/bestranking.csv"
        
        with open(source_file, 'r') as file_to_read:
            lines_1 = file_to_read.read()

        # Append the contents to the destination CSV file
        with open(destination_file, 'a') as dest_csv:
            dest_csv.write((lines_1))
        
    #checks if file exists
    print(os.path.exists(f'{target_dir}/bestranked_ligands_{target}.csv'))

    
    print("conda activate prc")
    for i in range (N_cores):
        filter_name = str("title='*_conf_01'")
        #print("obabel -imol2 " + target_dir + "docking"+str(i+1)+"/results"+str(i+1)+"/docked_ligands.mol2 -osdf -O "+target_dir+"docking"+str(i+1)+"/results"+str(i+1)+"/docked_ligands.sdf")
        print("obabel "+target_dir+str(i+1)+"/results1"+"/docked_ligands.mol2 -omol2 -O "+target_dir+str(i+1)+'/'+"bestranked_ligands.mol2 --filter " +'"'+filter_name+'"')
    

In [52]:
concatenate_w_water()

True
conda activate prc
obabel /media/emilia/HDD/Documents/PL/p85/1/results1/docked_ligands.mol2 -omol2 -O /media/emilia/HDD/Documents/PL/p85/1/bestranked_ligands.mol2 --filter "title='*_conf_01'"
obabel /media/emilia/HDD/Documents/PL/p85/2/results1/docked_ligands.mol2 -omol2 -O /media/emilia/HDD/Documents/PL/p85/2/bestranked_ligands.mol2 --filter "title='*_conf_01'"
obabel /media/emilia/HDD/Documents/PL/p85/3/results1/docked_ligands.mol2 -omol2 -O /media/emilia/HDD/Documents/PL/p85/3/bestranked_ligands.mol2 --filter "title='*_conf_01'"
obabel /media/emilia/HDD/Documents/PL/p85/4/results1/docked_ligands.mol2 -omol2 -O /media/emilia/HDD/Documents/PL/p85/4/bestranked_ligands.mol2 --filter "title='*_conf_01'"
obabel /media/emilia/HDD/Documents/PL/p85/5/results1/docked_ligands.mol2 -omol2 -O /media/emilia/HDD/Documents/PL/p85/5/bestranked_ligands.mol2 --filter "title='*_conf_01'"
obabel /media/emilia/HDD/Documents/PL/p85/6/results1/docked_ligands.mol2 -omol2 -O /media/emilia/HDD/Documents/

In [53]:
water = True
#uno archivos de obabel
if water:
    
    destination_file = target_dir + "out_plants_"+target+".mol2"
    open(destination_file, 'w').close()

    for i in range(1, N_cores + 1):
        source_file = target_dir + str (i) +  "/bestranked_ligands.mol2"
        
        with open(source_file, 'r') as source_csv:
            lines_2 = source_csv.read()

        # Append the contents to the destination CSV file
        with open(destination_file, 'a') as dest_csv:
            dest_csv.write((lines_2))
            
    print(os.path.exists(destination_file))

True


In [4]:

subprocess.Popen(['obabel','-imol2', PL_dir  + target + '/' + 'out_plants_' + target + '.mol2', '-osdf', '-O', PL_dir + '/' + target + '/' + 'out_' + target + '_pl.sdf'])

<Popen: returncode: None args: ['obabel', '-imol2', '/media/emilia/HDD/Docum...>

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is CHEMBL49860.0 55_entry_00055_conf_01)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is ZINC000000003374.0 203_entry_00203_conf_01)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is ZINC000000022315.5 454_entry_00454_conf_01)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is ZINC000000024466.3 482_entry_00482_conf_01)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is ZINC000000012616.4 358_entry_00358_conf_01)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is CHEMBL263528.1 29_entry_00029_conf_01)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is ZINC000000028311.6 544_entry_00544_conf_01)

***

In [5]:
#ruta = 'Documents/PLANTS/esr1/results1/ranking/'
df=pd.read_csv(target_dir + 'bestranked_ligands_' + target + '.csv', sep=',', skiprows=1, header=None)
df = df [[0,1]] # me quedo con el nombre de la molecula y con el TOTAL SCORE
df

,0,1
0,CHEMBL1083023.0 3_entry_00003_conf_01,-95.09850
1,CHEMBL1083023.1 4_entry_00004_conf_01,-92.80340
2,CHEMBL568542.0 68_entry_00068_conf_01,-88.89750
3,CHEMBL568543.0 69_entry_00069_conf_01,-88.44940
4,ZINC000000000207.5 87_entry_00087_conf_01,-86.91120
...,...,...
4029,ZINC000238607729.0 4021_entry_00661_conf_01,-42.44430
4030,ZINC000005133192.0 3769_entry_00409_conf_01,-29.93590
4031,ZINC000000626952.0 3479_entry_00119_conf_01,-8.76416
4032,ZINC000712035026.0 4033_entry_00001_conf_01,-58.24100


In [6]:
dr=df.loc[df[0]=='LIGAND_ENTRY']
df=df.drop(list(dr.index))
df[0]=[int(df.iloc[i][0].split(' ')[1].split('_')[0]) for i in range(len(df))]
sleep(3)
df



,0,1
0,3,-95.09850
1,4,-92.80340
2,68,-88.89750
3,69,-88.44940
4,87,-86.91120
...,...,...
4029,4021,-42.44430
4030,3769,-29.93590
4031,3479,-8.76416
4032,4033,-58.24100


In [7]:
dict_ix_scores={int(df.iloc[i][0]):df.iloc[i][1] for i in range(len(df))}
dict_ix_scores

{3: np.float64(-95.0985),
 4: np.float64(-92.8034),
 68: np.float64(-88.8975),
 69: np.float64(-88.4494),
 87: np.float64(-86.9112),
 243: np.float64(-86.8727),
 40: np.float64(-85.9878),
 136: np.float64(-85.9566),
 66: np.float64(-85.3924),
 77: np.float64(-85.1834),
 105: np.float64(-85.116),
 91: np.float64(-85.0007),
 78: np.float64(-84.5903),
 55: np.float64(-84.2454),
 569: np.float64(-84.1151),
 440: np.float64(-83.9547),
 331: np.float64(-83.8049),
 202: np.float64(-83.6417),
 67: np.float64(-83.3979),
 284: np.float64(-83.3659),
 19: np.float64(-83.3463),
 70: np.float64(-82.6697),
 27: np.float64(-82.5555),
 61: np.float64(-82.5299),
 403: np.float64(-82.496),
 555: np.float64(-82.4448),
 74: np.float64(-82.3432),
 48: np.float64(-82.1871),
 377: np.float64(-82.1843),
 391: np.float64(-82.1246),
 30: np.float64(-81.9554),
 104: np.float64(-81.7643),
 203: np.float64(-81.4817),
 248: np.float64(-81.4645),
 158: np.float64(-81.3981),
 387: np.float64(-81.3325),
 546: np.float6

In [8]:
f=open( target_dir + 'out_' + target + '_pl.sdf', 'r') # lo abre en modo lectura (r)
lines=f.readlines() # cada elemento en la lista corresponde a una linea en el archivo, lee todas la lineas
ix=[i for i in range(len(lines)) if lines[i]=='$$$$\n'] #Esta línea crea una lista llamada ix que contiene las posiciones de las líneas que contienen $$$$ en el archivo SDF.
ix.insert(0,-1) #agrega en el primer lugar un menos un pq en esa linea se "encuntra" el primer $$$$
print(len(ix)) # va a dar uno mas que el numero de mol pq le agrego uno

Nmols=len(ix)-1
print('Number of molecules ',Nmols)
f.close()

4029
Number of molecules  4028


In [9]:
new_file=[]
for i in range(Nmols):
    mol=lines[ix[i]+1:ix[i+1]-1] # delimitar el inicio y el final de la molécula, lee las lineas en el medio y las guarda en una lista llamada mol
    IX=int(mol[0].split(' ')[1].split('_')[0]) # se extrae el n de mol y se guarda con el int
    mol.append('M  END\n') #se agrega pq cuanod se delimita mol, la ultima linea no esta incluida
    mol.append('>  <IX> \n')
    mol.append(str(IX)+' \n')
    mol.append('>  <Score_PL> \n')
    mol.append(str(dict_ix_scores[IX])+' \n')
    #mol.append('>  <rank> \n')
    #mol.append(str(ranking[IX])+' \n')
    mol.append(' \n')
    mol.append('$$$$\n') # se agrega pq no lo leo y guardo antes
    for line in mol: #line toma el valor de cada línea en la lista mol
        new_file.append(line)


In [10]:
f=open(target_dir + 'out_' + target + '_pl.sdf', 'w')
for line in new_file:
    f.write(line)
f.close()

Now the use of ICM is required to sort the molecules by molecule number
this is a script for then be run inm ICM
esto lo podemos correr directo desde la consola de ICM gui, y ahi vamos a visualizar la tabal con las columnas
Para saber como se llama real la columna, nos paramos arriba del nombre y hacemos click derecho

In [62]:
f=open('sort.icm','w')
f.write('#!/usr/bin/icm \n')
f.write('call _startup \n')
f.write('\n')
f.write('maxFileSizeMb=2500 \n')
f.write('read table mol \"' + target_dir + 'out_' + target + '_pl.sdf\" \n')
f.write('sort out_ranked_ligands_pl.IX \n')
f.write('write table mol compress out_' + target +'_pl.sdf' +' \"' +  target_dir+ '/out_' + target + '_pl.sdf\" \n')
#f.write('quit \n')
f.close()

In [63]:
subprocess.Popen([icm_command,'-g','sort.icm'])

<Popen: returncode: None args: ['/home/emilia/icm-browser-pro-3.9-4/icmbrows...>

 Info>  127 colors and 18 graphic modes read from /home/emilia/icm-browser-pro-3.9-4/icm.clr
Startup> Loading config file /home/emilia/.browser/config/icm.cfg ..
